# Responsible AI Audit and Bias Mitigation with AIF360

## 🔍 Overview
End-to-end fairness auditing and bias mitigation using IBM AIF360 on high-stakes classification datasets (including COMPAS). The notebook measures fairness across protected groups, applies mitigation strategies, and compares fairness/utility trade-offs with reproducible evaluation.

## 🎯 Objective
- Establish baseline fairness and predictive performance.
- Evaluate multiple fairness metrics across protected groups.
- Apply mitigation techniques and validate improvements while tracking model utility.

## 🧠 Methodology
- Dataset loading and preprocessing with AIF360 dataset utilities.
- Group definition (privileged vs. unprivileged) for protected attributes.
- Baseline training and metric computation (performance + fairness).
- Mitigation and re-evaluation using consistent evaluation protocols.

## 📊 Key Results
Original tables/prints/figures are preserved as closely as possible; narrative text is translated and cleaned for portfolio quality.

## 🛠 Technologies & Skills
Python, Jupyter, AIF360, fairness metrics, bias mitigation, classification evaluation, reproducible analysis.

## 🧩 Why This Project Matters
Responsible deployment of ML requires measurable fairness and documented trade-offs. This project shows a practical workflow to audit, mitigate, and report bias in a way that can be reused for governance and monitoring.


<center><h1> MCD - Ciencia de data Responsable </h1><center>
<center><h2> Project 6: AIF360 Toolkit<h2></center>

![AIF360](https://opengraph.githubassets.com/4b7ff9fccf4e6cafd9e59f42921b31a0f7f6696b009c8519e584be278094769d/Trusted-AI/AIF360)

In [ ]:
#Section note: code preserved; comments translated/cleaned for professional presentation.
%%capture
!pip install aif360['all']

In [ ]:
#Section note: code preserved; comments translated/cleaned for professional presentation.


# elementales

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

# sklearn

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, plot_roc_curve, recall_score, precision_score, f1_score, roc_auc_score, roc_curve

# aif360

from aif360.sklearn.datasets import fetch_german
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import BinaryLabelDataset, StandardDataset


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

This section runs the same fairness audit workflow on a credit risk dataset to validate generalization beyond COMPAS.

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

X, y = fetch_german()
X.head()

checking_status  duration  \
sex    age   foreign_worker                             
male   aged  yes                         <0       6.0   
female young yes                   0<=X<200      48.0   
male   aged  yes                no checking      12.0   
             yes                         <0      42.0   
             yes                         <0      24.0   

                                             credit_history  \
sex    age   foreign_worker                                   
male   aged  yes             critical/other existing credit   
female young yes                              existing paid   
male   aged  yes             critical/other existing credit   
             yes                              existing paid   
             yes                         delayed previously   

                                         purpose  credit_amount  \
sex    age   foreign_worker                                       
male   aged  yes                        radio/tv         1169.0   
female young yes                        radio/tv         5951.0   
male   aged  yes                       education         2096.0   
             yes             furniture/equipment         7882.0   
             yes                         new car         4870.0   

                               savings_status employment  \
sex    age   foreign_worker                                
male   aged  yes             no known savings        >=7   
female young yes                         <100     1<=X<4   
male   aged  yes                         <100     4<=X<7   
             yes                         <100     4<=X<7   
             yes                         <100     1<=X<4   

                             installment_commitment other_parties  \
sex    age   foreign_worker                                         
male   aged  yes                                4.0          none   
female young yes                                2.0          none   
male   aged  yes                                2.0          none   
             yes                                2.0     guarantor   
             yes                                3.0          none   

                             residence_since  ...   age  other_payment_plans  \
sex    age   foreign_worker                   ...                              
male   aged  yes                         4.0  ...  67.0                 none   
female young yes                         2.0  ...  22.0                 none   
male   aged  yes                         3.0  ...  49.0                 none   
             yes                         4.0  ...  45.0                 none   
             yes                         4.0  ...  53.0                 none   

                              housing existing_credits                 job  \
sex    age   foreign_worker                                                  
male   aged  yes                  own              2.0             skilled   
female young yes                  own              1.0             skilled   
male   aged  yes                  own              1.0  unskilled resident   
             yes             for free              1.0             skilled   
             yes             for free              2.0             skilled   

                            num_dependents  own_telephone foreign_worker  \
sex    age   foreign_worker                                                
male   aged  yes                       1.0            yes            yes   
female young yes                       1.0           none            yes   
male   aged  yes                       2.0           none            yes   
             yes                       2.0           none            yes   
             yes                       2.0           none            yes   

                                sex marital_status  
sex    age   foreign_worker                         
male   aged  yes               male         single  
female youn

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

df = pd.concat([X,y], axis=1).drop(columns=['sex',  'foreign_worker']).rename(columns={'age': 'age_numeric'}).reset_index()
df.head()

,sex,age,foreign_worker,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,...,property_magnitude,age_numeric,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,marital_status,credit-risk
0,male,aged,yes,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,single,good
1,female,young,yes,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,...,real estate,22.0,none,own,1.0,skilled,1.0,none,div/dep/mar,bad
2,male,aged,yes,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,single,good
3,male,aged,yes,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,single,good
4,male,aged,yes,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,single,bad


In [ ]:
df = df.rename(columns={'age': 'age_cat', 'age_numeric': 'age'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   sex                     1000 non-null   category
 1   age_cat                 1000 non-null   category
 2   foreign_worker          1000 non-null   category
 3   checking_status         1000 non-null   category
 4   duration                1000 non-null   float64 
 5   credit_history          1000 non-null   category
 6   purpose                 1000 non-null   category
 7   credit_amount           1000 non-null   float64 
 8   savings_status          1000 non-null   category
 9   employment              1000 non-null   category
 10  installment_commitment  1000 non-null   float64 
 11  other_parties           1000 non-null   category
 12  residence_since         1000 non-null   float64 
 13  property_magnitude      1000 non-null   category
 14  age                     1

This section runs the same fairness audit workflow on a credit risk dataset to validate generalization beyond COMPAS.

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   sex                     1000 non-null   category
 1   age_cat                 1000 non-null   category
 2   foreign_worker          1000 non-null   category
 3   checking_status         1000 non-null   category
 4   duration                1000 non-null   float64 
 5   credit_history          1000 non-null   category
 6   purpose                 1000 non-null   category
 7   credit_amount           1000 non-null   float64 
 8   savings_status          1000 non-null   category
 9   employment              1000 non-null   category
 10  installment_commitment  1000 non-null   float64 
 11  other_parties           1000 non-null   category
 12  residence_since         1000 non-null   float64 
 13  property_magnitude      1000 non-null   category
 14  age                     1

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

df['sex'] = df['sex'].map({'male': 1, 'female': 0})
df['age_cat'] = df['age_cat'].map({'aged': 1, 'young': 0})
df['foreign_worker'] = df['foreign_worker'].map({'no': 1, 'yes': 0})
df['credit-risk'] = df['credit-risk'].map({'good': 1, 'bad': 0})

In [ ]:
# Separamos X e Y.

X = df.loc[:, df.columns != 'credit-risk']
y = df.loc[:, df.columns == 'credit-risk']

In [ ]:
df1 = X.copy()

In [ ]:
# Obtenemos variables dummies

ignore = ['sex', 'age_cat', 'foreign_worker']
for catcol in df1.select_dtypes(include='category').columns:
  if catcol in ignore:
    pass
  else:
    dummies = pd.get_dummies(X[catcol])
    dummies.columns = [catcol+ '_' + x for x in dummies.columns]
    X = pd.concat([X, dummies], axis=1).drop(columns=[catcol])

X.head()

,sex,age_cat,foreign_worker,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,...,job_unemp/unskilled non res,job_unskilled resident,job_skilled,job_high qualif/self emp/mgmt,own_telephone_none,own_telephone_yes,marital_status_div/dep/mar,marital_status_div/sep,marital_status_mar/wid,marital_status_single
0,1,1,0,6.0,1169.0,4.0,4.0,67.0,2.0,1.0,...,0,0,1,0,0,1,0,0,0,1
1,0,0,0,48.0,5951.0,2.0,2.0,22.0,1.0,1.0,...,0,0,1,0,1,0,1,0,0,0
2,1,1,0,12.0,2096.0,2.0,3.0,49.0,1.0,2.0,...,0,1,0,0,1,0,0,0,0,1
3,1,1,0,42.0,7882.0,2.0,4.0,45.0,1.0,2.0,...,0,0,1,0,1,0,0,0,0,1
4,1,1,0,24.0,4870.0,3.0,4.0,53.0,2.0,2.0,...,0,0,1,0,1,0,0,0,0,1


In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=908)

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

X_train.drop(columns=['age'], inplace=True)
X_test.drop(columns=['age'], inplace=True)

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

df_train = X_train.copy()
df_train['credit-score'] = y_train

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
# Debemos definir nuestros groups privilegeds y unprivilegeds.

privileged_groups = [{'age_cat': 1}]
unprivileged_groups = [{'age_cat': 0}]

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

attributes_params = dict(protected_attribute_names=['age_cat'], label_names=['credit-score'])
dt_train = BinaryLabelDataset(df=df_train, **attributes_params)

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

metric_age_train = BinaryLabelDatasetMetric(dt_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
print(f'Train Statistical Parity Difference (age): {metric_age_train.mean_difference()}')

Train Statistical Parity Difference (age): -0.13278706925580863


In [ ]:
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer
import pprint

In [ ]:
MetricTextExplainer(metric_age_train).mean_difference()

'Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.13278706925580863'

In [ ]:
pprint.pp(MetricJSONExplainer(metric_age_train).disparate_impact())

('{"metric": "Disparate Impact", "message": "Disparate impact (probability of '
 'favorable outcome for unprivileged instances / probability of favorable '
 'outcome for privileged instances): 0.8178037886955184", '
 '"numPositivePredictionsUnprivileged": 90.0, "numUnprivileged": 151.0, '
 '"numPositivePredictionsPrivileged": 473.0, "numPrivileged": 649.0, '
 '"description": "Computed as the ratio of rate of favorable outcome for the '
 'unprivileged group to that of the privileged group.", "ideal": "The ideal '
 'value of this metric is 1.0 A value < 1 implies higher benefit for the '
 'privileged group and a value >1 implies a higher benefit for the '
 'unprivileged group."}')


This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

lr = LogisticRegressionCV(solver='liblinear', cv=10, random_state=908)
lr.fit(X_train, np.ravel(y_train))

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

In [ ]:
#Section note: code preserved; comments translated/cleaned for professional presentation.

y_pred_proba = lr.predict_proba(X_test)[:,1]
y_pred = y_pred_proba >= 0.5

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1: {f1_score(y_test, y_pred)}")

Accuracy: 0.735
Recall: 0.8613138686131386
F1: 0.8166089965397924


This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

def get_aif_metrics(attr, df, label_names):

  privileged_groups = [{attr: 1}]
  unprivileged_groups = [{attr: 0}]

  attributes_params = dict(protected_attribute_names=[attr], label_names=label_names)
  dt = BinaryLabelDataset(df=df, **attributes_params, favorable_label=1, unfavorable_label=0)

  metric = BinaryLabelDatasetMetric(dt,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)

  return metric, dt

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
metric_age_train, train_aifdf = get_aif_metrics('age_cat', df_train, ['credit-score'])
print(f'Train Statistical Parity Difference (age): {metric_age_train.mean_difference()}')
print(f'Train Disparate Impact (age): {metric_age_train.disparate_impact()}')

Train Statistical Parity Difference (age): -0.13278706925580863
Train Disparate Impact (age): 0.8178037886955184


In [ ]:
y_train_pred_proba = lr.predict_proba(X_train)[:,1]
y_train_pred = y_train_pred_proba >= 0.5

In [ ]:
df_pred = X_train.copy()
df_pred['credit-score'] = y_train_pred

In [ ]:
metric_age_pred, pred_aifdf = get_aif_metrics('age_cat', df_pred, ['credit-score'])
print(f'Pred Statistical Parity Difference (age): {metric_age_pred.mean_difference()}')
print(f'Pred Disparate Impact (age): {metric_age_pred.disparate_impact()}')

Pred Statistical Parity Difference (age): -0.18487943754528113
Pred Disparate Impact (age): 0.7710176431929628


In [ ]:
orig_vs_pred = ClassificationMetric(train_aifdf, pred_aifdf,
                                                    unprivileged_groups=unprivileged_groups,
                                                    privileged_groups=privileged_groups)

In [ ]:
print(f'Error rate difference (unprivileged error rate - privileged error rate)= {orig_vs_pred.error_rate_difference()}\n')

print(f'False negative rate for privileged groups = {orig_vs_pred.false_negative_rate(privileged=True)}')
print(f'False negative rate for unprivileged groups = {orig_vs_pred.false_negative_rate(privileged=False)}')
print(f'False negative rate ratio = {orig_vs_pred.false_negative_rate_ratio()}\n')

print(f'False positive rate for privileged groups = {orig_vs_pred.false_positive_rate(privileged=True)}')
print(f'False positive rate for unprivileged groups = {orig_vs_pred.false_positive_rate(privileged=False)}')
print(f'False positive rate ratio = {orig_vs_pred.false_positive_rate_ratio()}')

Error rate difference (unprivileged error rate - privileged error rate)= 0.10803171460933281

False negative rate for privileged groups = 0.07188160676532769
False negative rate for unprivileged groups = 0.2222222222222222
False negative rate ratio = 3.0915032679738563

False positive rate for privileged groups = 0.48295454545454547
False positive rate for unprivileged groups = 0.39344262295081966
False positive rate ratio = 0.8146576663452266


This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

This section applies **Reject Option Classification** (post-processing) to adjust predictions near the decision boundary to improve fairness.

In [ ]:
lr

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
y_pred_proba = lr.predict_proba(X_test)[:,1]
y_pred = y_pred_proba >= 0.5

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.735


This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.
df_test_pred = X_test.copy()
df_test_pred['credit-score'] = y_pred

# Section note: code preserved; comments translated/cleaned for professional presentation.
df_test = X_test.copy()
df_test['credit-score'] = y_test

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
metric_age_test_pred, test_pred_aifdf = get_aif_metrics('age_cat', df_test_pred, ['credit-score'])
_, test_aifdf = get_aif_metrics('age_cat', df_test, ['credit-score'])

('{"metric": "Mean Difference", "message": "Mean difference (mean label value '
 'on unprivileged instances - mean label value on privileged instances): '
 '-0.30705526357700275", "numPositivesUnprivileged": 20.0, '
 '"numInstancesUnprivileged": 39.0, "numPositivesPrivileged": 132.0, '
 '"numInstancesPrivileged": 161.0, "description": "Computed as the difference '
 'of the rate of favorable outcomes received by the unprivileged group to the '
 'privileged group.", "ideal": "The ideal value of this metric is 0.0"}')


In [ ]:
cm_pred_test = ClassificationMetric(test_aifdf, test_pred_aifdf,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

In [ ]:
cm_pred_test.equal_opportunity_difference()

-0.24743589743589745

This section evaluates and applies **Equalized Odds**-style constraints and compares fairness/utility trade-offs.

In [ ]:
from aif360.algorithms.postprocessing.eq_odds_postprocessing import EqOddsPostprocessing

In [ ]:
roc = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups,
                           seed = 103)
roc.fit(test_aifdf, test_pred_aifdf)

In [ ]:
df = roc.predict(test_pred_aifdf)

In [ ]:
privileged_groups = [{'age_cat': 1}]
unprivileged_groups = [{'age_cat': 0}]


metric = BinaryLabelDatasetMetric(df,
                                  unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups)

In [ ]:
cm_pred_test_post = ClassificationMetric(test_aifdf, df,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

In [ ]:
cm_pred_test_post.equal_opportunity_difference()

0.007692307692307665

This section applies **Reweighing** (pre-processing) to reduce bias by adjusting instance weights across protected groups.

In [ ]:
lr

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

In [ ]:
df_train.head()

,sex,age_cat,foreign_worker,duration,credit_amount,installment_commitment,residence_since,existing_credits,num_dependents,checking_status_<0,...,job_unskilled resident,job_skilled,job_high qualif/self emp/mgmt,own_telephone_none,own_telephone_yes,marital_status_div/dep/mar,marital_status_div/sep,marital_status_mar/wid,marital_status_single,credit-score
589,1,1,0,12.0,2246.0,3.0,3.0,2.0,1.0,1,...,0,1,0,1,0,0,0,0,1,0
336,0,0,0,13.0,2101.0,2.0,4.0,1.0,1.0,0,...,1,0,0,1,0,1,0,0,0,1
446,0,1,0,36.0,1842.0,4.0,4.0,1.0,1.0,1,...,0,1,0,0,1,1,0,0,0,0
997,1,1,0,12.0,804.0,4.0,4.0,1.0,1.0,0,...,0,1,0,1,0,0,0,0,1,1
917,1,1,0,6.0,14896.0,1.0,4.0,1.0,1.0,1,...,0,0,1,0,1,0,0,0,1,0


In [ ]:
y_train_pred_proba = lr.predict_proba(X_train)[:,1]
y_train_pred = y_train_pred_proba >= 0.5

In [ ]:
df_pred = X_train.copy()
df_pred['credit-score'] = y_train_pred

In [ ]:
metric_age_train, train_aifdf = get_aif_metrics('age_cat', df_train, ['credit-score'])
print(f'Statistical Parity Difference (age): {metric_age_train.mean_difference()}')
print(f'Disparate Impact (age): {metric_age_train.disparate_impact()}')

Statistical Parity Difference (age): -0.13278706925580863
Disparate Impact (age): 0.8178037886955184


In [ ]:
metric_age_pred, pred_aifdf = get_aif_metrics('age_cat', df_pred, ['credit-score'])
print(f'Pred Statistical Parity Difference (age): {metric_age_pred.mean_difference()}')
print(f'Pred Disparate Impact (age): {metric_age_pred.disparate_impact()}')

Pred Statistical Parity Difference (age): -0.18487943754528113
Pred Disparate Impact (age): 0.7710176431929628


This section applies **Reweighing** (pre-processing) to reduce bias by adjusting instance weights across protected groups.

In [ ]:
from aif360.algorithms.preprocessing import Reweighing

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

train_aifdf.instance_weights[0:30]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

This section documents dataset preparation, model training, fairness evaluation, and mitigation using AIF360.

In [ ]:
# Debemos definir nuestros groups privilegeds y unprivilegeds.

privileged_groups = [{'age_cat': 1}]
unprivileged_groups = [{'age_cat': 0}]

# Section note: code preserved; comments translated/cleaned for professional presentation.

reweight = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

# Section note: code preserved; comments translated/cleaned for professional presentation.

reweighted = reweight.fit_transform(train_aifdf)

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

reweighted.instance_weights[0:30]

array([1.09242188, 1.18073611, 1.09242188, 0.96561047, 1.09242188,
       0.96561047, 1.09242188, 1.18073611, 0.73334016, 0.96561047,
       0.96561047, 0.96561047, 0.96561047, 0.96561047, 0.96561047,
       1.09242188, 0.96561047, 0.96561047, 1.09242188, 0.96561047,
       0.96561047, 0.96561047, 0.73334016, 0.96561047, 0.96561047,
       0.96561047, 1.09242188, 0.96561047, 0.96561047, 0.96561047])

In [ ]:
type(reweight) == type(Reweighing(privileged_groups=[], unprivileged_groups=[]))

True

In [ ]:
type(Reweighing)

aif360.decorating_metaclass.factory.<locals>.ApplyDecoratorMeta

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

reweighted_metrics = BinaryLabelDatasetMetric(reweighted,
                                              unprivileged_groups=unprivileged_groups,
                                              privileged_groups=privileged_groups)

In [ ]:
print(f'Reweighted Train Statistical Parity Difference (age): {reweighted_metrics.mean_difference()}')

Reweighted Train Statistical Parity Difference (age): -3.3306690738754696e-16


In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

lr = LogisticRegressionCV(solver='liblinear', cv=10, random_state=908)

# Section note: code preserved; comments translated/cleaned for professional presentation.

lr.fit(X_train, np.ravel(y_train), sample_weight=reweighted.instance_weights)

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

In [ ]:
# Section note: code preserved; comments translated/cleaned for professional presentation.

y_pred_proba = lr.predict_proba(X_test)[:,1]
y_pred = y_pred_proba >= 0.5

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.715


In [ ]:
df_test_rw = X_test.copy()
df_test_rw['credit-score'] = y_pred

In [ ]:
metric_age_test_rw, rw_test_pred_aifdf = get_aif_metrics('age_cat', df_test_rw, ['credit-score'])
print(f'Reweighted: Test Statistical Parity Difference (age): {metric_age_test_rw.mean_difference()}')
print(f'Reweighted: Test Disparate Impact (age): {metric_age_test_rw.disparate_impact()}')

Reweighted: Test Statistical Parity Difference (age): 0.02707437490046183
Reweighted: Test Disparate Impact (age): 1.032051282051282


In [ ]:
cm_pred_test_rw = ClassificationMetric(test_aifdf, rw_test_pred_aifdf,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

In [ ]:
cm_pred_test_rw.equal_opportunity_difference()

0.04401709401709397